# PyLASCA
a tool for ...

## Complete Area Alignment
Analyse and compare areas of LaSCA and mechanical pain detection

by

Sebastian Malkusch

2020-06-16

Institute for clinical pharmacology,

data science division,

Goethe-University Frankfurt,

Germany

malkusch@med.uni-frankfurt.de

www.smb.uni-frankfurt.de

http://share.smb.uni-frankfurt.de

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from os import walk
import os.path
from pyLASCA.source import fileNames

## Get Filenames

In [2]:
def get_filenames(folder_name):
    file_list = []
    for (dirpath, dirnames, filenames) in walk(folder_name):
        file_list.extend(filenames)
        break

    for filename in file_list:
        if not filename.endswith(".csv"):
            file_list.remove(filename)
    return(file_list)

## Define Application array

In [3]:
def define_application_sequence():
    application_array = ['i.c.', 'topic']
    proband_id = np.arange(1,20,1)
    application_day_1 = ['i.c.' for i in range(19)] 
    application_day_2 = ['topic' for i in range(19)]
    folder_name = "/Users/malkusch/PowerFolders/LaSCA/mechanic"
    file_name = folder_name + "/" +"sctcaps_01-19_Datensatz.xlsx"
    for i in range(19):
        probandId = str("sctcaps%s" % (str(i+1).zfill(2)))
        df = pd.read_excel(io = file_name, sheet_name = probandId)
        if(application_array[0] == df["start"][0]):
            application_day_1[i] = application_array[0]
            application_day_2[i] = application_array[1]
        else:
            application_day_1[i] = application_array[1]
            application_day_2[i] = application_array[0]
    df_appl = pd.DataFrame({'Id' : proband_id,
                            'day_1': application_day_1,
                           'day_2': application_day_2})
    return(df_appl)
        

## Create dataframe

In [4]:
folder_name = "/Users/malkusch/PowerFolders/LaSCA/area"
file_names = get_filenames(folder_name)

file_name = folder_name + "/" + file_names[0]
df_data = pd.read_csv(file_name, index_col=0)
for i in range(1, len(file_names),1):
    file_name = folder_name + "/" + file_names[i]
    df_data = df_data.append(pd.read_csv(file_name, index_col=0), ignore_index = True)
df_data = df_data.sort_values(by=['Id', 'day', 'effect', 'time_[min]', 'frame'], ascending = True, ignore_index = True)

df_appl = define_application_sequence()

application_array = ['i.c.', 'topic']
appl_type_array = []
for i, row_data in df_data.iterrows():
    row_appl = df_appl[df_appl['Id'] == row_data['Id']]
    if (row_data['day'] == 1):
        appl_type_array.append(row_appl['day_1'].iloc[0])
    elif (row_data['day'] == 2):
        appl_type_array.append(row_appl['day_2'].iloc[0])
    else:
        print("unexpected day")
        appl_type_array.append(np.nan)

df_data['application_type'] = appl_type_array

In [33]:
l = df_data.shape[0]
pxlSize_bl = np.zeros(l)*np.nan
median_bl = np.zeros(l)*np.nan
mean_bl = np.zeros(l)*np.nan
std_bl = np.zeros(l)*np.nan
min_bl = np.zeros(l)*np.nan
max_bl = np.zeros(l)*np.nan

df_bl = df_data[(df_data["time_[min]"] == -15) & (df_data["frame"] == 0)]
for i, row_data in df_data.iterrows():
    row_id = row_data['Id']
    row_day = row_data['day']
    row_bl = df_bl[(df_bl['Id'] == row_id) & (df_bl['day'] == row_day)]
    pxlSize_bl[i] = row_bl['pxlSize_[mm]'].iloc[0]
    median_bl[i] = row_bl['median_[adu]'].iloc[0]
    mean_bl[i] = row_bl['mean_[adu]'].iloc[0]
    std_bl[i] = row_bl['std_[adu]'].iloc[0]
    min_bl[i] = row_bl['min_[adu]'].iloc[0]
    max_bl[i] = row_bl['max_[adu]'].iloc[0]

df_data['pxlSize_bl_[mm]'] = pxlSize_bl
df_data['median_bl_[adu]'] = median_bl
df_data['mean_bl_[adu]'] = mean_bl
df_data['std_bl_[adu]'] = std_bl
df_data['min_bl_[adu]'] = min_bl
df_data['max_bl_[adu]'] = max_bl
df_data.head()

,Id,day,time_[min],frame,effect,pxlSize_[mm],median_[adu],mean_[adu],std_[adu],min_[adu],max_[adu],thr_[adu],application_type,pxlSize_bl_[mm],median_bl_[adu],mean_bl_[adu],std_bl_[adu],min_bl_[adu],max_bl_[adu]
0,1,1,-15,0,allodynia,0.240843,53.0,53.579980,14.698054,3,122,NaN,i.c.,0.240843,53.0,53.57998,14.698054,3.0,122.0
1,1,1,15,0,allodynia,0.240996,83.0,95.048264,52.776741,1,394,141.0,i.c.,0.240843,53.0,53.57998,14.698054,3.0,122.0
2,1,1,30,0,allodynia,0.239641,66.0,84.250268,52.547091,4,449,83.0,i.c.,0.240843,53.0,53.57998,14.698054,3.0,122.0
3,1,1,45,0,allodynia,0.241215,47.0,62.142570,41.926996,8,352,68.0,i.c.,0.240843,53.0,53.57998,14.698054,3.0,122.0
4,1,1,60,0,allodynia,0.240013,43.0,47.219715,23.450396,1,282,44.0,i.c.,0.240843,53.0,53.57998,14.698054,3.0,122.0


## Log Results

In [34]:
fn = fileNames.FileNames()
fn.folderName = "/Users/malkusch/PowerFolders/LaSCA/thr_alpha"
fn.baseName = "SCTCAPS"
fn.supplementalInformation = str("%s" %("thr_complete"))
fn.suffix = "csv"
fn.updateDateString()
fn.mergeFileName()

df_data.to_csv(path_or_buf = fn.outFileName)
print("wrote df to %s" %(fn.outFileName))

removed instance of FileNames from heap.
wrote df to /Users/malkusch/PowerFolders/LaSCA/thr_alpha/SCTCAPS_thr_complete_200821.csv
